# Initialize Packages and Load Dataset

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import torch
torch.cuda.empty_cache()

In [ ]:
from sklearn.model_selection import KFold
from datasets import load_dataset, DatasetDict, Dataset, concatenate_datasets
import datasets
import pandas as pd
import os
import logging
import nltk
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from random import sample
import random

In [ ]:
from datasets import load_from_disk

dataset = load_from_disk('data/decomposed/decomposed_test')

# Check Test Dataset + Add Tokenizer Function

In [ ]:
dataset

# Reduce it for testing
random_indices = random.sample(range(len(dataset)), 5)
dataset = dataset.select(random_indices)

In [ ]:
def generate_predictions(examples, tokenizer, model):
    generated_texts = []
    for example in examples:
        
        # Intial tokenization
        input_text = f"query:  {example['query']} answer: {example['answers']} header: {' '.join(map(str, example['table'].get('header', [])))} rows: {' '.join(map(str, example['table'].get('rows', [])))} title: {' '.join(map(str, example['table'].get('title', [])))}"
        input_ids = tokenizer.encode(input_text, return_tensors="pt")
        
        # Generate text and decode
        output_sequences = model.generate(input_ids)
        generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
        
        # Add to list of generated text
        generated_texts.append(generated_text)
    
    return generated_texts

# Load in models

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import BartTokenizer, BartForConditionalGeneration

# gpt2
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained("gpt2")
model_gpt2 = GPT2Model.from_pretrained("gpt2")

# t5 small
tokenizer_t5 = T5Tokenizer.from_pretrained("t5-small")
model_t5 = T5ForConditionalGeneration.from_pretrained("t5-small")

# flan t5
tokenizer_flant5 = T5Tokenizer.from_pretrained("google/flan-t5-small")
model_flant5 = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")google/flan-t5-large"

# Bart
tokenizer_bart = BartTokenizer.from_pretrained("facebook/bart-base")
model_bart = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

In [ ]:
models_and_tokenizers_with_names = [
    ("GPT2", tokenizer_gpt2, model_gpt2),
    ("T5 Small", tokenizer_t5, model_t5),
    ("FLAN-T5 Small", tokenizer_flant5, model_flant5),
    ("BART Base", tokenizer_bart, model_bart)
]

# Make predictions using each Model on Test Data

In [ ]:
for name, tokenizer, model in models_and_tokenizers_with_names:
    print(f"Model: {name}")
    predictions = generate_predictions(dataset, tokenizer, model)
    for i, prediction in enumerate(predictions):
        print(f"Example {i + 1}: {prediction}")
    print("\n" + "-"*50 + "\n")